<a href="https://colab.research.google.com/github/kevinwanghdat/Bayes/blob/main/Test_for_FA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Final Test

1. Pre work

In [2]:
# check required libraries are installed if not calling system to install
import sys
import subprocess
import pkg_resources

required = {'numpy', 'pandas', 'plotnine', 'matplotlib', 'seaborn', 
            'grid', 'shap', 'scikit-learn'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    print('Installing: ', missing)
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)
# delete unwanted variables
del required 
del installed 
del missing

Installing:  {'grid', 'shap'}


In [3]:
# Mount Google Drive
# We do not need to run this cell if you are not running this notebook in Google Colab

if 'google.colab' in str(get_ipython()):
    from google.colab import drive # import drive from Gogle colab
    root = '/content/drive'     # default location for the drive
    # print(root)                 # print content of ROOT (Optional)
    drive.mount(root)
else:
    print('Not running on CoLab')

Mounted at /content/drive


In [4]:
from pathlib import Path

if 'google.colab' in str(get_ipython()):
    # EDIT THE PROJECT PATH IF DIFFERENT WITH YOUR ONE
    project_path = Path(root) / 'MyDrive' / 'HDAT9500' / 'final-assignment'

    # OPTIONAL - set working directory according to your google drive project path
    # import os
    # Change directory to the location defined in project_path
    # os.chdir(project_path)
else:
    project_path = Path()

In [5]:
########## confusion matrix

def plot_confusion_matrix(confusion_matrix):
  # visualise the confusion matrix
  labels = ['No', 'Yes']
  ax = plt.subplot()
  sns.heatmap(confusion_matrix, annot = True, fmt = '.0f', ax = ax, cmap = 'viridis')

  # labels, titles and ticks
  ax.set_xlabel('Predicted labels')
  ax.set_ylabel('True labels')
  ax.set_title('Confusion Matrix')
  ax.xaxis.set_ticklabels(labels)
  ax.yaxis.set_ticklabels(labels)

In [6]:
# package import
import pickle
import numpy as np
import pandas as pd

---
# Question 1: Docstring

Create a docstring with:
1. The final aim of our program (50 words limit).
2. The variables and constants that you will use in this exercise (data dictionary). It is expected that you choose informative variable names and document your program (both docstrings and comments). 
3. Divide the docstring in sections. One section per question.
---

---
# Question 2: Read and check the `pickle` provided. Prepare the data so they can be used by the algorithms that you are going to create.
---

In [7]:
# import data
pickle_path = project_path/'data'/'hospital_data_final_assignment.pickle'


with open(pickle_path, 'rb') as data:
    hospital = pickle.load(data)
    


In [8]:
# check data
hospital.head()

,admission_id,patient_id,los,Age,number_diagnoses,num_lab_procedures,num_procedures,num_medications,number_emergency,number_inpatient,...,admission_source_grouped_Other,admission_source_grouped_Physician Referral,admission_source_grouped_Transfer from SNF,admission_source_grouped_Transfer from another health care facility,admission_type_grouped_Elective,admission_type_grouped_Emergency,admission_type_grouped_Not Available/Null,admission_type_grouped_Trauma Centre,admission_type_grouped_Urgent,readmission
0,62781276,360576,2,79,9,38,0,12,0,0,...,0,0,0,0,1,0,0,0,0,no
1,49223976,8151831,5,59,8,49,0,16,0,0,...,0,0,0,0,1,0,0,0,0,no
2,210634308,90219069,2,33,5,62,0,15,1,0,...,0,0,0,0,1,0,0,0,0,no
3,163620402,54294840,6,42,9,77,0,30,0,0,...,0,0,0,0,1,0,0,0,0,no
4,69055020,1634688,1,62,7,13,5,6,0,0,...,0,0,0,0,1,0,0,0,0,no


In [9]:
hospital = hospital.drop(['admission_id', 'patient_id'], axis = 1)
hospital.head()

,los,Age,number_diagnoses,num_lab_procedures,num_procedures,num_medications,number_emergency,number_inpatient,number_outpatient,sex_Female,...,admission_source_grouped_Other,admission_source_grouped_Physician Referral,admission_source_grouped_Transfer from SNF,admission_source_grouped_Transfer from another health care facility,admission_type_grouped_Elective,admission_type_grouped_Emergency,admission_type_grouped_Not Available/Null,admission_type_grouped_Trauma Centre,admission_type_grouped_Urgent,readmission
0,2,79,9,38,0,12,0,0,0,1,...,0,0,0,0,1,0,0,0,0,no
1,5,59,8,49,0,16,0,0,0,0,...,0,0,0,0,1,0,0,0,0,no
2,2,33,5,62,0,15,1,0,1,1,...,0,0,0,0,1,0,0,0,0,no
3,6,42,9,77,0,30,0,0,0,1,...,0,0,0,0,1,0,0,0,0,no
4,1,62,7,13,5,6,0,0,0,0,...,0,0,0,0,1,0,0,0,0,no


In [10]:
hospital.describe(include='all') 



,los,Age,number_diagnoses,num_lab_procedures,num_procedures,num_medications,number_emergency,number_inpatient,number_outpatient,sex_Female,...,admission_source_grouped_Other,admission_source_grouped_Physician Referral,admission_source_grouped_Transfer from SNF,admission_source_grouped_Transfer from another health care facility,admission_type_grouped_Elective,admission_type_grouped_Emergency,admission_type_grouped_Not Available/Null,admission_type_grouped_Trauma Centre,admission_type_grouped_Urgent,readmission
count,69267.000000,69267.000000,69267.000000,69267.000000,69267.000000,69267.000000,69267.000000,69267.000000,69267.000000,69267.000000,...,69267.000000,69267.000000,69267.000000,69267.000000,69267.000000,69267.000000,69267.000000,69267.000000,69267.000000,69267
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57348
mean,6.155413,67.281028,7.620382,43.111698,1.618375,16.062974,0.282963,0.221014,0.774886,0.531869,...,0.001429,0.228464,0.005183,0.046559,0.350354,0.374377,0.105115,0.000245,0.169908,NaN
std,5.240195,16.149325,2.326673,19.925355,1.866568,8.377290,0.883817,0.650029,2.022099,0.498987,...,0.037779,0.419846,0.071806,0.210694,0.477084,0.483965,0.306704,0.015664,0.375554,NaN
min,1.000000,20.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,2.000000,56.000000,6.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
50%,4.000000,69.000000,8.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
75%,9.000000,79.000000,9.000000,57.000000,3.000000,20.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,NaN


In [11]:
# Sanity check
# Display at least 100 columns, so we can have a proper look into every column.
pd.set_option('display.max_columns', 100)
display(hospital[:5])
hospital.shape

,los,Age,number_diagnoses,num_lab_procedures,num_procedures,num_medications,number_emergency,number_inpatient,number_outpatient,sex_Female,sex_Male,max_glu_serum_>200,max_glu_serum_>300,max_glu_serum_None,max_glu_serum_Norm,A1Cresult_>7,A1Cresult_>8,A1Cresult_None,A1Cresult_Norm,group_name_1_Blood_&_immune,group_name_1_CNS,group_name_1_Cancer,group_name_1_Cardiac_&_circulatory,group_name_1_Digestive,group_name_1_Endocrine,group_name_1_Infectious,group_name_1_Mental_&_Substance,group_name_1_Other,group_name_1_Respiratory,group_name_2_Blood_&_immune,group_name_2_CNS,group_name_2_Cancer,group_name_2_Cardiac_&_circulatory,group_name_2_Digestive,group_name_2_Endocrine,group_name_2_Infectious,group_name_2_Mental_&_Substance,group_name_2_Other,group_name_2_Respiratory,group_name_3_Blood_&_immune,group_name_3_CNS,group_name_3_Cancer,group_name_3_Cardiac_&_circulatory,group_name_3_Digestive,group_name_3_Endocrine,group_name_3_Infectious,group_name_3_Mental_&_Substance,group_name_3_Other,group_name_3_Respiratory,discharge_disposition_grouped_Discharged to home,discharge_disposition_grouped_Home health service,discharge_disposition_grouped_Left AMA,discharge_disposition_grouped_Not available/Null,discharge_disposition_grouped_Other,discharge_disposition_grouped_Short term hospital,discharge_disposition_grouped_Transferred to ICF,discharge_disposition_grouped_Transferred to SNF,admission_source_grouped_Clinic Referral,admission_source_grouped_Emergency Room,admission_source_grouped_Not available/Null,admission_source_grouped_Other,admission_source_grouped_Physician Referral,admission_source_grouped_Transfer from SNF,admission_source_grouped_Transfer from another health care facility,admission_type_grouped_Elective,admission_type_grouped_Emergency,admission_type_grouped_Not Available/Null,admission_type_grouped_Trauma Centre,admission_type_grouped_Urgent,readmission
0,2,79,9,38,0,12,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,no
1,5,59,8,49,0,16,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,no
2,2,33,5,62,0,15,1,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,no
3,6,42,9,77,0,30,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,no
4,1,62,7,13,5,6,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,no


(69267, 70)

In [12]:
hospital.columns

Index(['los', 'Age', 'number_diagnoses', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_emergency',
       'number_inpatient', 'number_outpatient', 'sex_Female', 'sex_Male',
       'max_glu_serum_>200', 'max_glu_serum_>300', 'max_glu_serum_None',
       'max_glu_serum_Norm', 'A1Cresult_>7', 'A1Cresult_>8', 'A1Cresult_None',
       'A1Cresult_Norm', 'group_name_1_Blood_&_immune', 'group_name_1_CNS',
       'group_name_1_Cancer', 'group_name_1_Cardiac_&_circulatory',
       'group_name_1_Digestive', 'group_name_1_Endocrine',
       'group_name_1_Infectious', 'group_name_1_Mental_&_Substance',
       'group_name_1_Other', 'group_name_1_Respiratory',
       'group_name_2_Blood_&_immune', 'group_name_2_CNS',
       'group_name_2_Cancer', 'group_name_2_Cardiac_&_circulatory',
       'group_name_2_Digestive', 'group_name_2_Endocrine',
       'group_name_2_Infectious', 'group_name_2_Mental_&_Substance',
       'group_name_2_Other', 'group_name_2_Respiratory',
       

In [13]:
# Write Python code here for 'percentages':
hospital['readmission'].value_counts(normalize=True)*100

no     82.792672
yes    17.207328
Name: readmission, dtype: float64

In [14]:
#### PASS NO NULL
#### Total number of null records of our dataset
print("Total number of null records: ", hospital.isnull().values.sum())

# Null values of all our features
print(hospital.isnull().sum())

Total number of null records:  0
los                                          0
Age                                          0
number_diagnoses                             0
num_lab_procedures                           0
num_procedures                               0
                                            ..
admission_type_grouped_Emergency             0
admission_type_grouped_Not Available/Null    0
admission_type_grouped_Trauma Centre         0
admission_type_grouped_Urgent                0
readmission                                  0
Length: 70, dtype: int64


In [15]:

from sklearn.model_selection import train_test_split

X = hospital.drop(['readmission'], axis = 1)
y = hospital['readmission'].values
y_binary = [0 if x=='no' else 1 for x in y]
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size = 0.2, train_size = 0.8, stratify = y, random_state = 42)

In [16]:
# Sanity Checks:
print('******************************************')
print('y - Yes values =', sum(i =='yes' for i in y))
print('y - No values =', sum(i =='no' for i in y))
print('******************************************\n')

# Create y_binary
y_binary = [0 if x=='no' else 1 for x in y]

# Sanity Checks:
print('\n******************************************')
print('y_binary - 1 values =', sum(i ==1 for i in y_binary))
print('y_binary - 0 values =', sum(i ==0 for i in y_binary))
print('******************************************')

******************************************
y - Yes values = 11919
y - No values = 57348
******************************************


******************************************
y_binary - 1 values = 11919
y_binary - 0 values = 57348
******************************************


In [17]:
y

array(['no', 'no', 'no', ..., 'no', 'no', 'no'], dtype=object)

---
## Question 4: Training, and hyper-parameter tuning of the Logistic Regression model.  

Hyper-parameters: 

- `C` values: 10, 100.
- `class_weight`: only two pairs. Choose one of these two combinations and explain why.

    * A weight of (80% for class 1, 20% for class 0) and (70% for class 1, 30% for class 0)
    * A weight of (80% for class 0, 20% for class 1) and (70% for class 0, 30% for class 1). 

- `penalty` values: l1, l2.
- 3-fold cross-validation for the grid search.
- `f1` as the score to choose the best model in the grid search.
- `n_jobs`=-1.
- do not include the heatmaps in the final submission.
- do not change these hyper-parameters, 
- keep the remaining set of hyper-parameters in the default state. 
---

In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import GridSearchCV
scaler = StandardScaler()


In [20]:
class_weight_dict= [{0:0.2, 1:0.8}, {0:0.3, 1:0.7}]
Log_Reg_1 = LogisticRegression(solver='liblinear')
pipe = Pipeline([('Transform', scaler), ('Estimator', Log_Reg_1)])
param_grid = {'Estimator__C':[10, 100], 'Estimator__penalty':['l1', 'l2'], 
              'Estimator__class_weight' : class_weight_dict}


# I changed the scoring to f1, since the dataset was not balanced.
grid_search = GridSearchCV(pipe, param_grid, cv = 3, scoring='f1', n_jobs=-1)




In [21]:
grid_search.fit(X_train, y_train)


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('Transform', StandardScaler()),
                                       ('Estimator',
                                        LogisticRegression(solver='liblinear'))]),
             n_jobs=-1,
             param_grid={'Estimator__C': [10, 100],
                         'Estimator__class_weight': [{0: 0.2, 1: 0.8},
                                                     {0: 0.3, 1: 0.7}],
                         'Estimator__penalty': ['l1', 'l2']},
             scoring='f1')

In [22]:

print("Best hyper-parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.4f}".format(grid_search.best_score_))

Best hyper-parameters: {'Estimator__C': 10, 'Estimator__class_weight': {0: 0.3, 1: 0.7}, 'Estimator__penalty': 'l2'}
Best cross-validation score: 0.9323
